# Secrets
This notebook is used to test code for working with the secrets directory and for generating SAS tokens when necessary. Some of this has or will be moved into the ooiod library in this repo.

## Create secret pickle file

In [ ]:
import ooiod
import datetime

In [ ]:
keys = ooiod.secrets.load_keys('../secrets/tjcrone.pickle')

In [ ]:
ooiod.secrets.write_pickle(['camhd', 'botpt'], keys['ooiopendata'], datetime.timedelta(days=2),
                           '../secrets/daxsoule.pickle', overwrite=True)

## Load keys from secret pickle file

In [ ]:
import ooiod

In [ ]:
keys = ooiod.secrets.load_keys('../secrets/daxsoule.pickle')